In [ ]:
import pickle
import os
os.getcwd()

#htmlを解析しデータを抽出するためのライブラリ
from bs4 import BeautifulSoup
#httpリクエストを行うためのライブラリ
#webページの情報を取得する
import requests
# 時間稼ぎ用
import time

import pandas as pd
df_all = pd.read_pickle("df2001_2023.pkl")

# 血統登録番号カラムを文字列型に変換
df_all['血統登録番号'] = df_all['血統登録番号'].astype(str)

# 血統登録番号の文字数に応じた処理を関数で定義

# 6桁：文字列2000を前に連結
# 7桁：文字列200を前に連結
# 8桁：文字列20を前に連結

def process_blood(x):
    if len(x) == 6:
        return '2000' + x
    elif len(x) == 7:
        return '200' + x
    elif len(x) == 8:
        return '20' + x
    else:
        return x  # 他のケースでは元の値を返す

# '血統登録番号'カラムの値を加工
#  applyメソッドは引数へ与えた関数を各値に処理して返す
df_all['血統登録番号'] = df_all['血統登録番号'].astype(str).apply(process_blood)

#血統登録番号の文字列の長さごとの内容を再確認
for i in range(10,11):
    invalid_blood_rows = df_all[df_all['血統登録番号'].str.len() == i]
    print(f"{i}文字の内容：{invalid_blood_rows['血統登録番号'].unique()}")
    print(f"{i}文字の要素の数＜一意＞：{len(invalid_blood_rows['血統登録番号'].unique())}")
    
# ユニークな血統登録番号をリストに格納
blood_id = df_all['血統登録番号'].unique()
#len(blood_id)

In [ ]:
del df_all

In [19]:
del Aptitude_data_list

NameError: name 'Aptitude_data_list' is not defined

In [ ]:
%%time

# スクレイピング結果を格納するリスト
Aptitude_data_list = []

# 血統番号で馬番号をループする
for id in blood_id[20001:len(blood_id)+1]:
    #len(blood_id+1)

    # URLの設定
    url = "https://db.netkeiba.com/horse/" + id + "/"

    # HTTPリクエストを行うための準備
    response = requests.get(url)

    # エンコーディング設定
    response.encoding = 'euc-jp'

    # HTMLの作成
    html = response.text

    # BeautifulSoupパーサーの作成
    soup = BeautifulSoup(html, "html.parser")

    # 探したい`<th>`タグのテキストのリスト
    search_texts = set(['コース適性', '距離適性', '脚質', '成長', '重馬場'])

    # スクレイピング結果を格納するリスト
    results = []

    # HTML属性の解析および値の抽出
    # `<dl class="tekisei">`を見つける
    tekisei_dl = soup.find('dl', class_='tekisei')

    if tekisei_dl:
        # `<dl>`内の`<tr>`タグをループ処理
        for tr in tekisei_dl.find_all('tr'):

            th = tr.find('th')

            if th and th.text in search_texts:

                # `<tr>`タグ内の`<td>`タグから`<img>`タグを探す
                width_attrs = [img['width'] for img in tr.find_all('img', width=True)]

                # 二番目の`width`属性の値を取得して保存
                if len(width_attrs) >= 2:
                    results.append(width_attrs[1])

    Aptitude_data_list.append({id: results})

    # 時間稼ぎ
    time.sleep(2.2)

# 辞書型をpickle形式で保存
with open("data20001_LAST.pickle", "wb") as f:
    pickle.dump(Aptitude_data_list, f)

len(Aptitude_data_list)

In [ ]:
len(Aptitude_data_list)

In [23]:
len(blood_id)+1

108830